In [1]:
import numpy as np
import scipy
import numpy 
import matplotlib.pyplot as plt
from scipy import linalg, special, stats
from numpy import genfromtxt
import ML_support as ml

In [2]:
class TiedCovClassifier:
    def __init__(self):
        self.mu = {}

    def train(self, DTR, LTR):
        self.mu, C = ml.MU_Cov_calculator(DTR, LTR)
        N = DTR.shape[1]
        self.C_ = numpy.zeros((DTR.shape[0], DTR.shape[0]))
        for i in numpy.unique(LTR):
            Nc = DTR[:, LTR == i].shape[1]
            self.C_ += Nc*C[i]
        self.C_ /= N


    def test(self, DTE, LTE):
        S = numpy.zeros((numpy.unique(LTE).size, DTE.shape[1]))
        predicted = []

        for i in numpy.unique(LTE):
            S[i, :] = ml.GAU_logpdf_ND(DTE, self.mu[i], self.C_) + numpy.log(1 / 2)

        Sp = scipy.special.logsumexp(S, axis=0)

        for x, p in zip(S.T, Sp):
            tmp = x - p
            predicted.append(numpy.argmax(tmp))

        predicted = numpy.array(predicted)

        True_prediction = numpy.array([predicted == LTE])

        error = 1 - (numpy.count_nonzero(True_prediction) / True_prediction.size)

        print("TiedCovClassifier error:", error)
        
        return error

In [3]:
Data, label = ml.loadFile('../Train.txt')
(DTR, LTR), (DTE,LTE) = ml.split_db_2to1(Data, label, seed=42)

In [4]:
G=TiedCovClassifier()
G.train(DTR,LTR)
G.test(DTE,LTE)

TiedCovClassifier error: 0.023849512932482386


In [5]:
Gn=TiedCovClassifier()
Gn.train(ml.z_normalization(DTR), LTR)
Gn.test(DTE, LTE)
Gn.test(ml.z_normalization(DTE), LTE)

TiedCovClassifier error: 0.9287873698354048
TiedCovClassifier error: 0.023513604299630453


In [6]:
GG=TiedCovClassifier()
GG.train(ml.gaussianize(DTR), LTR)
GG.test(DTE, LTE)
GG.test(ml.gaussianize(DTE), LTE)

TiedCovClassifier error: 0.9076251259657373
TiedCovClassifier error: 0.060127645280483755


In [14]:
class TiedCovClassifier_mod1:
    def __init__(self):
        self.mu = {}

    def train(self, DTR, LTR):
        self.mu, C = ml.MU_Cov_calculator(DTR, LTR)
        N = DTR.shape[1]
        self.C_ = numpy.zeros((DTR.shape[0], DTR.shape[0]))
        for i in numpy.unique(LTR):
            Nc = DTR[:, LTR == i].shape[1]
            self.C_ += Nc*C[i]
        self.C_ /= N


    def test(self, DTE, LTE, app, ROC=False):
        S = numpy.zeros((numpy.unique(LTE).size, DTE.shape[1]))
        ll = numpy.zeros((numpy.unique(LTE).size, DTE.shape[1]))
        predicted = []

        for i in numpy.unique(LTE):
            ll[i, :] = ml.GAU_logpdf_ND(DTE, self.mu[i], self.C_)

        llr = numpy.array(ll[1, :]-ll[0, :])
        
        CM = ml.compute_optimal_B_decision(app, llr, LTE)
        
        app_bayes_risk=ml.compute_Bayes_risk(CM, app)
        DCF = ml.compute_norm_Bayes(app_bayes_risk, app)
        
        minDCF= ml.compute_min_DCF(llr, app, LTE)
        error = 1-(CM[0, 0]+CM[1,1])/(len(LTE))
        if(ROC == True):
            ml.plot_ROC(app, llr, LTE)
        
        print("\-/ \-/ \-/ \-/ \-/ ")
        print("Gaussian Classifier error:", error)
        print(app,"DCF:", DCF, "minDCF:", minDCF)
        print('CM\n', CM)
        print("/-\ /-\ /-\ /-\ /-\ ")

In [8]:
G=TiedCovClassifier_mod1()
G.train(DTR,LTR)
G.test(DTE,LTE, [1/2,1,1])

\-/ \-/ \-/ \-/ \-/ 
Gaussian Classifier error: 0.023849512932482386
[0.5, 1, 1] DCF: 0.1993890047776058 minDCF: 0.10910840454881907
/-\ /-\ /-\ /-\ /-\ 


In [10]:
G.test(DTE,LTE, [0.1,1,1])

\-/ \-/ \-/ \-/ \-/ 
Gaussian Classifier error: 0.023849512932482386
[0.1, 1, 1] DCF: 0.2526828611802705 minDCF: 0.2426606554067694
/-\ /-\ /-\ /-\ /-\ 


In [11]:
G.test(DTE,LTE, [0.7,1,1])

\-/ \-/ \-/ \-/ \-/ 
Gaussian Classifier error: 0.023849512932482386
[0.7, 1, 1] DCF: 0.45635870174730264 minDCF: 0.20784424556445277
/-\ /-\ /-\ /-\ /-\ 


In [13]:
G.test(DTE,LTE, [0.6,1,1])

\-/ \-/ \-/ \-/ \-/ 
Gaussian Classifier error: 0.023849512932482386
[0.6, 1, 1] DCF: 0.2957526411412421 minDCF: 0.15026848798869524
/-\ /-\ /-\ /-\ /-\ 


In [24]:
def kfold(classifier, D, L, fold, app= None):
    error = 0
    N = int(D.shape[1]/fold) #numero di elementi per ogni fold
    numpy.random.seed(0) #imposto il seed del generatore di numeri casuali -> in tal modo genererò la stessa sequenza di numeri casuali aventi seed uguale
    indexes = numpy.random.permutation(D.shape[1]) #genero una sequenza di numeri casuali che vanno da 0 al num_di_campioni
    errors = []
    DTR=[]
    LTR=[]
    DTE=[]
    LTE=[]
    for j in range(fold):
        test_indexes = indexes[(j*N):((j+1)*N)] #selezioni gli indici che identificano i campioni (casuali) del test set
        if(j > 0): #se il test set non è preso dalla prima fold (--> il test set è una fold intermedia o l'ultima fold)
            left_indexes = indexes[0:(j*N)] #allora prendo tutti gli indici che stanno a sinistra di tale fold
        else: #se il test set è preso dalla prima fold
            right_indexes = indexes[((j+1)*N):] #prendo tutti gli indici a destra della prima fold

        if(j == 0): #se il test set è preso dalla prima fold
            train_indexes = right_indexes #assegno agli indici di training quelli che stanno a destra della prima fold
        elif(j == fold-1): #se il test set è preso dall'ultima fold
            train_indexes = left_indexes #assegno agli indici di training quelli che stanno a sinistra dell'ultima fold
        else: #in questo caso il test set è preso da una fold intermedia
            train_indexes = numpy.hstack((left_indexes, right_indexes)) #pertanto assegno agli indici di training quelli appartenenti alle fold di sinistra e di destra

        DTR.append(D[:, train_indexes])  #definisco insieme di training e di testing
        LTR.append(L[train_indexes])
        DTE.append(D[:, test_indexes])
        LTE.append(L[test_indexes])
        classifier.train(DTR[j], LTR[j])
        if(app != None):
            classifier.test(DTE[j],LTE[j], app)
        else:
            pass
    #    errors.append(classifier.test(DTE[j],LTE[j]))
    #for j in range(fold):
     #   print("error fold number",j,":", errors[j])
    #min = numpy.argmin(errors)
    #return (DTR[min], LTR[min]), (DTE[min], LTE[min])
    return (DTR, LTR), (DTE, LTE)

In [12]:
kg=TiedCovClassifier()
(DTR, LTR), (DTE, LTE) = kfold(kg, Data, label, 5)

TiedCovClassifier error: 0.024089635854341762
TiedCovClassifier error: 0.020728291316526648
TiedCovClassifier error: 0.0212885154061625
TiedCovClassifier error: 0.0212885154061625
TiedCovClassifier error: 0.02352941176470591


In [25]:
kgg=TiedCovClassifier_mod1()
(DTR, LTR), (DTE, LTE) = kfold(kgg, Data, label, 5, [1/2, 1,1])

\-/ \-/ \-/ \-/ \-/ 
Gaussian Classifier error: 0.024089635854341762
[0.5, 1, 1] DCF: 0.19729833485292 minDCF: 0.12922884975286722
CM
 [[1595.   35.]
 [   8.  147.]]
/-\ /-\ /-\ /-\ /-\ 
\-/ \-/ \-/ \-/ \-/ 
Gaussian Classifier error: 0.020728291316526648
[0.5, 1, 1] DCF: 0.1911006670709521 minDCF: 0.10028805336567617
CM
 [[1637.   25.]
 [  12.  111.]]
/-\ /-\ /-\ /-\ /-\ 
\-/ \-/ \-/ \-/ \-/ 
Gaussian Classifier error: 0.0212885154061625
[0.5, 1, 1] DCF: 0.17900093562447228 minDCF: 0.12523675863170625
CM
 [[1613.   28.]
 [  10.  134.]]
/-\ /-\ /-\ /-\ /-\ 
\-/ \-/ \-/ \-/ \-/ 
Gaussian Classifier error: 0.0212885154061625
[0.5, 1, 1] DCF: 0.20069502527701089 minDCF: 0.07138997653642036
CM
 [[1613.   33.]
 [   5.  134.]]
/-\ /-\ /-\ /-\ /-\ 
\-/ \-/ \-/ \-/ \-/ 
Gaussian Classifier error: 0.02352941176470591
[0.5, 1, 1] DCF: 0.1849889766476166 minDCF: 0.10761146428647875
CM
 [[1600.   31.]
 [  11.  143.]]
/-\ /-\ /-\ /-\ /-\ 
